In [ ]:
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
import textwrap
import logging

logging.basicConfig(level=logging.INFO)

class SimplifiedContentGenerator:
    def __init__(self, template_path: str):
        self.prs = Presentation(template_path)
        self._validate_template()
        self.LINE_HEIGHT = Pt(9)
        self.CHARS_PER_ROW = 55
        self.WHITE = RGBColor(255, 255, 255)
        
    def _validate_template(self):
        """Validate template contains required shapes"""
        for slide in self.prs.slides[:2]:
            if not any(shape.name == "coSummaryShape" for shape in slide.shapes):
                raise ValueError("coSummaryShape missing in template")

    def parse_markdown(self, md_content: str) -> list:
        """Clean markdown content into plain text lines"""
        cleaned_lines = []
        for line in md_content.strip().split('\n'):
            clean_line = line.strip().lstrip('#*- ').replace('**', '').replace('__', '')
            if clean_line:
                wrapper = textwrap.TextWrapper(width=self.CHARS_PER_ROW)
                cleaned_lines.extend(wrapper.wrap(clean_line))
        return cleaned_lines

    def _format_paragraph(self, paragraph, text, is_continuation=False):
        """Apply strict formatting to paragraphs"""
        try:
            pf = paragraph.paragraph_format
            pf.left_indent = Inches(0.07)
            pf.space_before = Pt(0)
            pf.space_after = Pt(0)
            pf.line_spacing = 1.0
            
            run = paragraph.add_run()
            run.text = f"(continued) {text}" if is_continuation else text
            run.font.name = 'Arial'
            run.font.bold = True
            run.font.size = Pt(9)
            run.font.color.rgb = self.WHITE
        except AttributeError as e:
            logging.error(f"Formatting error: {str(e)}")
            raise

    def _calculate_max_lines(self, shape):
        """Dynamic line calculation based on shape height"""
        return int(shape.height / self.LINE_HEIGHT) - 1

    def remove_excess_slides(self, used_slides):
        """Correctly remove unused slides via XML manipulation"""
        xml_slides = self.prs.slides._sldIdLst  # Access XML slide list
        
        # Remove slides in reverse order to prevent index shifting
        for slide_idx in range(len(xml_slides)-1, used_slides-1, -1):
            if slide_idx >= len(xml_slides):
                continue
                
            # Get XML slide element and its relationship ID
            slide_element = xml_slides[slide_idx]
            rId = slide_element.rId  # Correct rId access from XML element
            
            # Remove relationship and XML element
            self.prs.part.drop_rel(rId)
            xml_slides.remove(slide_element)
            logging.info(f"Removed slide {slide_idx + 1}")


    def generate(self, md_content: str, output_path: str):
        """Main generation method with error handling"""
        try:
            lines = self.parse_markdown(md_content)
            current_slide = 0
            line_idx = 0
            
            while line_idx < len(lines) and current_slide < 2:
                slide = self.prs.slides[current_slide]
                shape = next(s for s in slide.shapes if s.name == "coSummaryShape")
                tf = shape.text_frame
                tf.clear()
                
                # Initialize first paragraph
                if not tf.paragraphs:
                    tf.add_paragraph()
                para = tf.paragraphs[0]
                
                remaining_space = self._calculate_max_lines(shape)
                
                while line_idx < len(lines) and remaining_space > 0:
                    # Add continuation marker if needed
                    if current_slide > 0 and line_idx == 0:
                        self._format_paragraph(para, lines[line_idx], is_continuation=True)
                    else:
                        self._format_paragraph(para, lines[line_idx])
                    
                    line_idx += 1
                    remaining_space -= 1
                    
                    if remaining_space > 0 and line_idx < len(lines):
                        para = tf.add_paragraph()
                
                current_slide += 1
            
            # Remove unused slides
            self._remove_excess_slides(current_slide)
            self.prs.save(output_path)
            logging.info(f"Generated {current_slide} slides successfully")
            
        except Exception as e:
            logging.error(f"Generation failed: {str(e)}")
            raise

# Sample markdown content
sample_md_content = """
# Executive Summary
This comprehensive analysis provides key insights into current market conditions and strategic recommendations for the upcoming fiscal quarter.

## Key Performance Indicators
Revenue growth has exceeded expectations by 15% compared to previous quarter, driven by strong performance in technology and healthcare sectors.

## Strategic Recommendations
Prioritize digital transformation initiatives to maintain competitive advantage and improve operational efficiency across all business units.

## Risk Assessment
Current market volatility requires careful monitoring and adaptive strategy implementation to balance challenges and opportunities.
"""

# Usage example
if __name__ == "__main__":
    try:
        generator = SimplifiedContentGenerator("template.pptx")
        generator.generate(sample_md_content, "summary_report.pptx")
        print("Generation completed successfully!")
    except Exception as e:
        logging.error(f"Fatal error: {str(e)}")
